<a href="https://colab.research.google.com/github/newtubes/datasets_AI/blob/main/fine_tuningRonron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🌟 **PROYECTO RONRÓN - Fine-Tuning con DeepSeek-R1**
# @markdown 💜 *By Rebeca Romcy*

# 1. INSTALAR DEPENDENCIAS
!pip install -q transformers torch accelerate sentencepiece datasets peft pandas bitsandbytes

# 2. CONFIGURAR MODELO (ajustado para GPU T4)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1"  # Modelo de 236B (usamos adaptadores)

# Configuración 4-bit para ahorrar memoria
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. CARGAR DATOS (formato Ronrón)
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_json('/content/ronron_dataset.jsonl', lines=True))
dataset = dataset.train_test_split(test_size=0.1)

def format_ronron(example):
    return f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {example['prompt']}
### Respuesta: {example['completion']}"""

tokenized_dataset = dataset.map(
    lambda x: tokenizer(format_ronron(x), truncation=True, padding="max_length", max_length=256),
    batched=True
)

# 4. CONFIGURAR LoRA (para fine-tuning eficiente)
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# 5. ENTRENAR (ajustado para DeepSeek)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ronron_deepseek",
    per_device_train_batch_size=2,  # Reduce batch por memoria
    gradient_accumulation_steps=4,  # Compensa batch pequeño
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

print("🚀 ¡Entrenando con magia DeepSeek!")
trainer.train()

# 6. GUARDAR Y PROBAR
model.save_pretrained("/content/ronron_deepseek_ajustado")

def generar_respuesta(pregunta):
    input_text = f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {pregunta}
### Respuesta:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba
print("\n💖 **Respuesta de Ronrón:**")
print(generar_respuesta("¿Qué hago cuando me siento triste sin razón?"))

RuntimeError: No GPU or XPU found. A GPU or XPU is needed for FP8 quantization.

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

In [ ]:
# @title 🌟 **PROYECTO RONRÓN - Fine-Tuning con DeepSeek-R1**
# @markdown 💜 *By Rebeca Romcy*

# 1. INSTALAR DEPENDENCIAS
!pip install -q transformers torch accelerate sentencepiece datasets peft pandas bitsandbytes

# 2. CONFIGURAR MODELO (ajustado para GPU T4)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1"  # Modelo de 236B (usamos adaptadores)

# Configuración 4-bit para ahorrar memoria
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. CARGAR DATOS (formato Ronrón)
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_json('/content/ronron_dataset.jsonl', lines=True))
dataset = dataset.train_test_split(test_size=0.1)

def format_ronron(example):
    return f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {example['prompt']}
### Respuesta: {example['completion']}"""

tokenized_dataset = dataset.map(
    lambda x: tokenizer(format_ronron(x), truncation=True, padding="max_length", max_length=256),
    batched=True
)

# 4. CONFIGURAR LoRA (para fine-tuning eficiente)
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# 5. ENTRENAR (ajustado para DeepSeek)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ronron_deepseek",
    per_device_train_batch_size=2,  # Reduce batch por memoria
    gradient_accumulation_steps=4,  # Compensa batch pequeño
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

print("🚀 ¡Entrenando con magia DeepSeek!")
trainer.train()

# 6. GUARDAR Y PROBAR
model.save_pretrained("/content/ronron_deepseek_ajustado")

def generar_respuesta(pregunta):
    input_text = f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {pregunta}
### Respuesta:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba
print("\n💖 **Respuesta de Ronrón:**")
print(generar_respuesta("¿Qué hago cuando me siento triste sin razón?"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.85M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

configuration_deepseek.py:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-R1:
- configuration_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_deepseek.py:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-R1:
- modeling_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


ValueError: FP8 quantized models is only supported on GPUs with compute capability >= 8.9 (e.g 4090/H100), actual = `7.5`

In [ ]:
# @title 🌟 **PROYECTO RONRÓN - Fine-Tuning con DeepSeek-R1**
# @markdown 💜 *By Rebeca Romcy*

# 1. INSTALAR DEPENDENCIAS
!pip install -q transformers torch accelerate sentencepiece datasets peft pandas bitsandbytes

# 2. CONFIGURAR MODELO (ajustado para GPU T4)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1"  # Modelo de 236B (usamos adaptadores)

# Configuración 4-bit para ahorrar memoria
# Adjusted BitsAndBytesConfig for wider GPU compatibility
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # Removed specific 4-bit parameters that might require higher compute capability
)


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. CARGAR DATOS (formato Ronrón)
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_json('/content/ronron_dataset.jsonl', lines=True))
dataset = dataset.train_test_split(test_size=0.1)

def format_ronron(example):
    return f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {example['prompt']}
### Respuesta: {example['completion']}"""

tokenized_dataset = dataset.map(
    lambda x: tokenizer(format_ronron(x), truncation=True, padding="max_length", max_length=256),
    batched=True
)

# 4. CONFIGURAR LoRA (para fine-tuning eficiente)
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# 5. ENTRENAR (ajustado para DeepSeek)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ronron_deepseek",
    per_device_train_batch_size=2,  # Reduce batch por memoria
    gradient_accumulation_steps=4,  # Compensa batch pequeño
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

print("🚀 ¡Entrenando con magia DeepSeek!")
trainer.train()

# 6. GUARDAR Y PROBAR
model.save_pretrained("/content/ronron_deepseek_ajustado")

def generar_respuesta(pregunta):
    input_text = f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {pregunta}
### Respuesta:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba
print("\n💖 **Respuesta de Ronrón:**")
print(generar_respuesta("¿Qué hago cuando me siento triste sin razón?"))

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


ValueError: FP8 quantized models is only supported on GPUs with compute capability >= 8.9 (e.g 4090/H100), actual = `7.5`

In [ ]:
# @title 🌟 **PROYECTO RONRÓN - Fine-Tuning con DeepSeek-R1**
# @markdown 💜 *By Rebeca Romcy*

# 1. INSTALAR DEPENDENCIAS
!pip install -q transformers torch accelerate sentencepiece datasets peft pandas bitsandbytes

# 2. CONFIGURAR MODELO (ajustado para GPU T4)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1"  # Modelo de 236B (usamos adaptadores)

# Configuración 4-bit para ahorrar memoria
# Adjusted BitsAndBytesConfig for wider GPU compatibility
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # Removed specific 4-bit parameters that might require higher compute capability
)


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. CARGAR DATOS (formato Ronrón)
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_json('/content/ronron_dataset.jsonl', lines=True))
dataset = dataset.train_test_split(test_size=0.1)

def format_ronron(example):
    return f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {example['prompt']}
### Respuesta: {example['completion']}"""

tokenized_dataset = dataset.map(
    lambda x: tokenizer(format_ronron(x), truncation=True, padding="max_length", max_length=256),
    batched=True
)

# 4. CONFIGURAR LoRA (para fine-tuning eficiente)
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# 5. ENTRENAR (ajustado para DeepSeek)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ronron_deepseek",
    per_device_train_batch_size=2,  # Reduce batch por memoria
    gradient_accumulation_steps=4,  # Compensa batch pequeño
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

print("🚀 ¡Entrenando con magia DeepSeek!")
trainer.train()

# 6. GUARDAR Y PROBAR
model.save_pretrained("/content/ronron_deepseek_ajustado")

def generar_respuesta(pregunta):
    input_text = f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {pregunta}
### Respuesta:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba
print("\n💖 **Respuesta de Ronrón:**")
print(generar_respuesta("¿Qué hago cuando me siento triste sin razón?"))

ValueError: FP8 quantized models is only supported on GPUs with compute capability >= 8.9 (e.g 4090/H100), actual = `7.5`

In [ ]:
# @title 🌟 **PROYECTO RONRÓN - Fine-Tuning con DeepSeek-R1**
# @markdown 💜 *By Rebeca Romcy*

# 1. INSTALAR DEPENDENCIAS
!pip install -q transformers torch accelerate sentencepiece datasets peft pandas bitsandbytes

# 2. CONFIGURAR MODELO (ajustado para GPU T4)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1"  # Modelo de 236B (usamos adaptadores)

# Configuración 4-bit para ahorrar memoria
# Adjusted BitsAndBytesConfig for wider GPU compatibility
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # Removed specific 4-bit parameters that might require higher compute capability
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float16
)


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 3. CARGAR DATOS (formato Ronrón)
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_json('/content/ronron_dataset.jsonl', lines=True))
dataset = dataset.train_test_split(test_size=0.1)

def format_ronron(example):
    return f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {example['prompt']}
### Respuesta: {example['completion']}"""

tokenized_dataset = dataset.map(
    lambda x: tokenizer(format_ronron(x), truncation=True, padding="max_length", max_length=256),
    batched=True
)

# 4. CONFIGURAR LoRA (para fine-tuning eficiente)
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# 5. ENTRENAR (ajustado para DeepSeek)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ronron_deepseek",
    per_device_train_batch_size=2,  # Reduce batch por memoria
    gradient_accumulation_steps=4,  # Compensa batch pequeño
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

print("🚀 ¡Entrenando con magia DeepSeek!")
trainer.train()

# 6. GUARDAR Y PROBAR
model.save_pretrained("/content/ronron_deepseek_ajustado")

def generar_respuesta(pregunta):
    input_text = f"""### Contexto: Eres Ronrón, un asistente mágico.
### Pregunta: {pregunta}
### Respuesta:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba
print("\n💖 **Respuesta de Ronrón:**")
print(generar_respuesta("¿Qué hago cuando me siento triste sin razón?"))

ValueError: FP8 quantized models is only supported on GPUs with compute capability >= 8.9 (e.g 4090/H100), actual = `7.5`